In [1]:
# env is ShapelyFiona
# Week 7 Homework, Part 2
# three shapefiles were sucessfully viewed in ArcGIS Pro after defining projection: WGS 1984 (EPSG 4326)


In [2]:
import fiona

In [3]:
import shapely

In [4]:
# import csv and print contents
import csv
with open('some.csv', 'r') as f: # the 'rb' parameter gave an error that it couldn't be read in as bytes
    reader = csv.DictReader(f)  # type(f) is _io.TextIOWrapper  # type(reader) is csv.DictReader
    for row in reader: # type(row) is <class 'collections.OrderedDict'>
        print(row)
        

OrderedDict([('name', 'Chicago'), ('lat', '41.88'), ('lon', '-87.63')])
OrderedDict([('name', 'Kansas City'), ('lat', '39.101'), ('lon', '-94.584')])


In [5]:
# create point objects from contents of file
import csv
from shapely.geometry import Point
with open('some.csv', 'r') as f:
    reader = csv.DictReader(f)   #type(reader) is csv.DictReader
    for row in reader:  # type(row) is <class 'collections.OrderedDict'>
        point = Point(float(row['lon']), float(row['lat']))  # type(point) is shapely.geometry.point.Point

In [6]:
# create a shapefile named some.shp, containing points
import csv
from shapely.geometry import Point, mapping
from fiona import collection


schema = { 'geometry': 'Point', 'properties': { 'name': 'str' } }
with collection(
    "some.shp", "w", "ESRI Shapefile", schema) as output:   # type(output) is <class 'fiona.collection.Collection'>
    with open('some.csv', 'r') as f: # type(f) is <class '_io.TextIOWrapper'>
        reader = csv.DictReader(f)   # type(reader) is <class 'csv.DictReader'>
        for row in reader:           # type(row) is <class 'collections.OrderedDict'>
            point = Point(float(row['lon']), float(row['lat']))    # type(point) is <class 'shapely.geometry.point.Point'>
            output.write({
                'properties': {
                    'name': row['name']      # type row['name'] is <class 'str'>
                },
                'geometry': mapping(point)   # type(mapping(point)) is <class 'dict'>
            })
        

In [7]:
# create a shapefile named some_buffer.shp
from shapely.geometry import mapping, shape
from fiona import collection

with collection("some.shp", "r") as input:  #type(input) is fiona.collection.Collection
    # input.schema is {'properties': OrderedDict([('name', 'str:80')]), 'geometry': 'Polygon'}
    # schema = input.schema.copy()
    schema = { 'geometry': 'Polygon', 'properties': { 'name': 'str' } }   #type(schema) is dict
    with collection(                              # type(collection) is function
        "some_buffer.shp", "w", "ESRI Shapefile", schema) as output:
        for point in input:   #type(point) is dict, type(input) is fiona.collection.Collection         
            # temp = (mapping(shape(point['geometry']).buffer(5.0)))

            # len(temp['coordinates'])) # is 1
            # len(temp['coordinates'][0]) # is 66
            # len(temp['coordinates'][0][0])) # is 2

            output.write({
                'properties': {
                    'name': point['properties']['name'] #type is str
                },
                'geometry': mapping(shape(point['geometry']).buffer(5.0)) 
                #type(mapping(shape(point['geometry']))) is dict
                #type(mapping(shape(point['geometry']).buffer(5.0))) is dict
                #type(shape(point['geometry'])) is shapely.geometry.point.Point
                #type(point['geometry']) is dict           
            })

In [8]:
# create a shapefile with name some_union.shp
from shapely.geometry import mapping, shape
from shapely.ops import cascaded_union
from fiona import collection

with collection("some_buffer.shp", "r") as input:
    schema = input.schema.copy()   #type(schema) is dict
    with collection(
            "some_union.shp", "w", "ESRI Shapefile", schema) as output:
            # output.schema is {'properties': OrderedDict([('name', 'str:80')]), 'geometry': 'Polygon'}
        shapes = []
        for f in input:   #type(input) is fiona.collection.Collection # type(f) is <class 'dict'>
            shapes.append(shape(f['geometry']))
            #type(shape(f['geometry'])) is shapely.geometry.polygon.Polygon
        merged = cascaded_union(shapes)  #type(merged) is shapely.geometry.polygon.Polygon
        output.write({
            'properties': {
                'name': 'Buffer Area'
                },
            'geometry': mapping(merged)  #type(mapping(merged)) is dict
            })